In [1]:
import time
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble
import xgboost
from keras.layers import *
from keras import layers, models, optimizers
from keras.preprocessing import text, sequence
from sklearn import preprocessing
from sklearn import naive_bayes
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
import pickle

Using TensorFlow backend.


In [2]:
# Load data
drive.mount('/content/drive', force_remount=True)
DATA_PATH = "/content/drive/My Drive/NLP"

x_data = pickle.load(open(DATA_PATH + '/Processed Data/x_data.pkl', 'rb'))
y_data = pickle.load(open(DATA_PATH + '/Processed Data/y_data.pkl', 'rb'))

x_test = pickle.load(open(DATA_PATH + '/Processed Data/x_test.pkl', 'rb'))
y_test = pickle.load(open(DATA_PATH + '/Processed Data/y_test.pkl', 'rb'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(x_data)

x_data_tfidf = tfidf_vect_ngram_char.transform(x_data)
x_test_tfidf = tfidf_vect_ngram_char.transform(x_test)

In [4]:
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(x_data_tfidf)

x_data_tfidf_svd = svd.transform(x_data_tfidf)
x_test_tfidf_svd = svd.transform(x_test_tfidf)

In [5]:
def train_model(classifier, x_data, y_data, x_test, y_test, is_neuralnet=False, n_epochs=100):
    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=42)

    if is_neuralnet:
        classifier.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=n_epochs, batch_size=512)

        val_predictions = classifier.predict(x_val)
        test_predictions = classifier.predict(x_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(x_train, y_train)

        train_predictions = classifier.predict(x_train)
        val_predictions = classifier.predict(x_val)
        test_predictions = classifier.predict(x_test)

    # Evaluation

    print("> Validation Accuracy: ", metrics.accuracy_score(val_predictions, y_val))

    print("> Test Accuracy: ", metrics.accuracy_score(test_predictions, y_test))

    print("> Validation Precision: ", metrics.precision_score(y_val, val_predictions, average='macro'))

    print("> Test Precision: ", metrics.precision_score(y_test, test_predictions, average='macro'))

    print("> Validation Recall: ", metrics.recall_score(y_val, val_predictions, average='macro'))

    print("> Test Recall: ", metrics.recall_score(y_test, test_predictions, average='macro'))

    print("> Validation F1 Score: ", metrics.f1_score(y_val, val_predictions, average='macro'))

    print("> Test F1 Score: ", metrics.f1_score(y_test, test_predictions, average='macro'))

In [ ]:
# Naive Bayes
# Naive Bayes (Multinomial)
print("\n\n----- Naive Bayes (Multinomial) -----")
nbm_start = time.time()
train_model(naive_bayes.MultinomialNB(), x_data_tfidf,
            y_data, x_test_tfidf, y_test, is_neuralnet=False)
nbm_stop = time.time()
print(f"> Training time: {nbm_stop - nbm_start}s")



----- Naive Bayes (Multinomial) -----
> Validation Accuracy:  0.7225198200128562
> Test Accuracy:  0.7022563103644185
> Validation Precision:  0.8181466290813446
> Test Precision:  0.7368080129860384
> Validation Recall:  0.6713667235894115
> Test Recall:  0.6579089117559855
> Validation F1 Score:  0.6815292515173849
> Test F1 Score:  0.6191425680030507
> Training time: 4.107416868209839s


In [ ]:
# Naive Bayes (Bernoulli)
print("\n\n----- Naive Bayes (Bernoulli) -----")
nbb_start = time.time()
train_model(naive_bayes.BernoulliNB(), x_data_tfidf, y_data,
            x_test_tfidf, y_test, is_neuralnet=False)

# train_model(naive_bayes.BernoulliNB(), x_data_tfidf_svd, y_data, x_test_tfidf_svd, y_test, is_neuralnet=False)
nbb_stop = time.time()
print(f"> Training time: {nbb_stop - nbb_start}s")



----- Naive Bayes (Bernoulli) -----
> Validation Accuracy:  0.6723805442468395
> Test Accuracy:  0.7004910315721127
> Validation Precision:  0.6784206291954972
> Test Precision:  0.6254306207618897
> Validation Recall:  0.6694802997990517
> Test Recall:  0.6893074567931239
> Validation F1 Score:  0.6661743207765388
> Test F1 Score:  0.6362987562411754
> Training time: 6.235400676727295s


In [ ]:
# Logistic Regression
print("\n\n----- Logistic Regression -----")
lr_start = time.time()
train_model(linear_model.LogisticRegression(), x_data_tfidf,
            y_data, x_test_tfidf, y_test, is_neuralnet=False)

# train_model(linear_model.LogisticRegression(), x_data_tfidf_svd, y_data, x_test_tfidf_svd, y_test, is_neuralnet=False)
lr_stop = time.time()
print(f"> Training time: {lr_stop - lr_start}s")



----- Logistic Regression -----


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


> Validation Accuracy:  0.8776515963145489
> Test Accuracy:  0.875517409301193
> Validation Precision:  0.8739744881869606
> Test Precision:  0.7999164383792159
> Validation Recall:  0.8707447286392451
> Test Recall:  0.8570216413153044
> Validation F1 Score:  0.8717427831929214
> Test F1 Score:  0.8177995718078823
> Training time: 210.40027117729187s


In [ ]:
# Support Vector Machine
print("\n\n----- Support Vector Machine -----")
svm_start = time.time()
train_model(svm.SVC(), x_data_tfidf_svd, y_data,
            x_test_tfidf_svd, y_test, is_neuralnet=False)
svm_stop = time.time()
print(f"> Training time: {svm_stop - svm_start}s")



----- Support Vector Machine -----
> Validation Accuracy:  0.8780801371330619
> Test Accuracy:  0.87490869247626
> Validation Precision:  0.8759719738723221
> Test Precision:  0.7984890139684064
> Validation Recall:  0.8698675726752144
> Test Recall:  0.8523340117469099
> Validation F1 Score:  0.8720318939102268
> Test F1 Score:  0.8154089551437216
> Training time: 1277.9954187870026s


In [ ]:
# Random Forest Classifier
print("\n\n----- Random Forest Classifier -----")
rfc_start = time.time()
train_model(ensemble.RandomForestClassifier(), x_data_tfidf_svd,
            y_data, x_test_tfidf_svd, y_test, is_neuralnet=False)
rfc_stop = time.time()
print(f"> Training time: {rfc_stop - rfc_start}s")



----- Random Forest Classifier -----
> Validation Accuracy:  0.8150846368116563
> Test Accuracy:  0.8014974433893353
> Validation Precision:  0.8169152818157275
> Test Precision:  0.7196826504048078
> Validation Recall:  0.7983230969560611
> Test Recall:  0.7706412897931714
> Validation F1 Score:  0.8045205717294974
> Test F1 Score:  0.7242619118549918
> Training time: 121.11324262619019s


In [ ]:
# XGBoost
print("\n\n----- XGBoost -----")
xgb_start = time.time()
train_model(xgboost.XGBClassifier(), x_data_tfidf_svd, y_data, x_test_tfidf_svd, y_test, is_neuralnet=False)
xgb_stop = time.time()
print(f"> Training time: {xgb_stop - xgb_start}s")



----- XGBoost -----
> Validation Accuracy:  0.7958002999785729
> Test Accuracy:  0.7825460595730866
> Validation Precision:  0.7883971184801566
> Test Precision:  0.6915379329441967
> Validation Recall:  0.7836612386815399
> Test Recall:  0.7592488396934096
> Validation F1 Score:  0.7849479249691171
> Test F1 Score:  0.7060594762635977
> Training time: 1073.7828052043915s


In [6]:
dnn_start = time.time()
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

In [ ]:
# Deep Neural Network
dnn_start = time.time()
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

def create_dnn_model():
    input_layer = Input(shape=(300,))
    layer = Dense(1024, activation='relu')(input_layer)
    layer = Dense(1024, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

classifier = create_dnn_model()
print("\n\n----- Deep Neural Network -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
dnn_stop = time.time()
print(f"> Training time: {dnn_stop - dnn_start}s")



----- Deep Neural Network -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 9s 218us/step - loss: 0.9620 - accuracy: 0.7073 - val_loss: 0.4541 - val_accuracy: 0.8539
Epoch 2/100
42001/42001 [==============================] - 9s 209us/step - loss: 0.4121 - accuracy: 0.8626 - val_loss: 0.3977 - val_accuracy: 0.8693
Epoch 3/100
42001/42001 [==============================] - 9s 207us/step - loss: 0.3589 - accuracy: 0.8790 - val_loss: 0.3981 - val_accuracy: 0.8663
Epoch 4/100
42001/42001 [==============================] - 9s 208us/step - loss: 0.3279 - accuracy: 0.8893 - val_loss: 0.3892 - val_accuracy: 0.8732
Epoch 5/100
42001/42001 [==============================] - 11s 274us/step - loss: 0.2859 - accuracy: 0.9033 - val_loss: 0.3963 - val_accuracy: 0.8633
Epoch 6/100
42001/42001 [==============================] - 9s 203us/step - loss: 0.2503 - accuracy: 0.9155 - val_loss: 0.3653 - val_accuracy: 0.8794
Epoch 7/100
42001/4200

In [ ]:
# Recurrent Neural Network - LSTM
def create_lstm_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = LSTM(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

lstm_start = time.time()
classifier = create_lstm_model()
print("\n\n----- Recurrent Neural Network - LSTM -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
lstm_stop = time.time()
print(f"> Training time: {lstm_stop - lstm_start}s")



----- Recurrent Neural Network - LSTM -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 12s 277us/step - loss: 2.3695 - accuracy: 0.1576 - val_loss: 2.0401 - val_accuracy: 0.2580
Epoch 2/100
42001/42001 [==============================] - 11s 260us/step - loss: 1.8571 - accuracy: 0.2954 - val_loss: 1.8231 - val_accuracy: 0.3302
Epoch 3/100
42001/42001 [==============================] - 11s 263us/step - loss: 1.6810 - accuracy: 0.3587 - val_loss: 1.6231 - val_accuracy: 0.3853
Epoch 4/100
42001/42001 [==============================] - 11s 256us/step - loss: 1.4726 - accuracy: 0.4457 - val_loss: 1.3917 - val_accuracy: 0.4834
Epoch 5/100
42001/42001 [==============================] - 11s 258us/step - loss: 1.1827 - accuracy: 0.5858 - val_loss: 1.1710 - val_accuracy: 0.5809
Epoch 6/100
42001/42001 [==============================] - 11s 266us/step - loss: 1.0397 - accuracy: 0.6483 - val_loss: 1.0749 - val_accuracy: 0.6315
Epoch

In [ ]:
# Recurrent Neural Network - GRU

def create_gru_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = GRU(128, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

gru_start = time.time()
classifier = create_gru_model()
print("\n\n----- Recurrent Neural Network - GRU -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
gru_stop = time.time()
print(f"> Training time: {gru_stop - gru_start}s")



----- Recurrent Neural Network - GRU -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 12s 281us/step - loss: 2.4864 - accuracy: 0.1193 - val_loss: 2.2899 - val_accuracy: 0.1680
Epoch 2/100
42001/42001 [==============================] - 14s 338us/step - loss: 1.9706 - accuracy: 0.2562 - val_loss: 1.8970 - val_accuracy: 0.2760
Epoch 3/100
42001/42001 [==============================] - 11s 273us/step - loss: 1.7550 - accuracy: 0.3369 - val_loss: 1.6972 - val_accuracy: 0.3535
Epoch 4/100
42001/42001 [==============================] - 12s 275us/step - loss: 1.6379 - accuracy: 0.3797 - val_loss: 1.9849 - val_accuracy: 0.2901
Epoch 5/100
42001/42001 [==============================] - 11s 262us/step - loss: 1.5956 - accuracy: 0.4036 - val_loss: 1.5810 - val_accuracy: 0.4176
Epoch 6/100
42001/42001 [==============================] - 11s 268us/step - loss: 1.5181 - accuracy: 0.4424 - val_loss: 1.5320 - val_accuracy: 0.4408
Epoch 

In [ ]:
# Bidirectional RNN

def create_brnn_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu'))(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)

    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

brnn_start = time.time()
classifier = create_brnn_model()
print("\n\n----- Bidirectional RNN -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
brnn_stop = time.time()
print(f"> Training time: {brnn_stop - brnn_start}s")



----- Bidirectional RNN -----
Train on 42001 samples, validate on 4667 samples
Epoch 1/100
42001/42001 [==============================] - 20s 483us/step - loss: 1.6623 - accuracy: 0.4809 - val_loss: 0.9460 - val_accuracy: 0.6812
Epoch 2/100
42001/42001 [==============================] - 20s 465us/step - loss: 0.8340 - accuracy: 0.7211 - val_loss: 0.7794 - val_accuracy: 0.7326
Epoch 3/100
42001/42001 [==============================] - 19s 460us/step - loss: 0.7301 - accuracy: 0.7567 - val_loss: 0.7475 - val_accuracy: 0.7489
Epoch 4/100
42001/42001 [==============================] - 20s 470us/step - loss: 0.6603 - accuracy: 0.7815 - val_loss: 0.7517 - val_accuracy: 0.7472
Epoch 5/100
42001/42001 [==============================] - 20s 465us/step - loss: 0.6234 - accuracy: 0.7926 - val_loss: 0.6029 - val_accuracy: 0.7992
Epoch 6/100
42001/42001 [==============================] - 21s 491us/step - loss: 0.5889 - accuracy: 0.8019 - val_loss: 0.5757 - val_accuracy: 0.8031
Epoch 7/100
42001/4

In [7]:
# Recurrent Convolutional Neural Network

def create_rcnn_model():
    input_layer = Input(shape=(300,))

    layer = Reshape((10, 30))(input_layer)
    layer = Bidirectional(GRU(128, activation='relu', return_sequences=True))(layer)
    layer = Convolution1D(100, 3, activation="relu")(layer)
    layer = Flatten()(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)

    output_layer = Dense(13, activation='softmax')(layer)

    classifier = models.Model(input_layer, output_layer)
    classifier.summary()
    classifier.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return classifier

rcnn_start = time.time()
classifier = create_rcnn_model()
print("\n\n----- Recurrent Convolutional Neural Network -----")
train_model(classifier=classifier, x_data=x_data_tfidf_svd, y_data=y_data_n, x_test=x_test_tfidf_svd, y_test=y_test_n, is_neuralnet=True, n_epochs=100)
rcnn_stop = time.time()
print(f"> Training time: {rcnn_stop - rcnn_start}s")

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 10, 30)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 10, 256)           122112    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 100)            76900     
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               410112    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               2626